In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import os
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

# Load environment variables (for API keys)
load_dotenv()

# Create sample fixed income market data
def generate_sample_data():
    # Create date range for the past 30 days
    end_date = datetime.now()
    start_date = end_date - timedelta(days=30)
    dates = pd.date_range(start=start_date, end=end_date, freq='B')  # Business days
    
    # Treasury yields
    np.random.seed(42)
    treasuries = {
        '2Y': 4.2 + np.random.normal(0, 0.05, len(dates)),
        '5Y': 4.5 + np.random.normal(0, 0.07, len(dates)),
        '10Y': 4.7 + np.random.normal(0, 0.08, len(dates)),
        '30Y': 4.8 + np.random.normal(0, 0.1, len(dates))
    }
    
    # Corporate bond spreads
    corp_spreads = {
        'AAA': 0.5 + np.random.normal(0, 0.03, len(dates)),
        'AA': 0.7 + np.random.normal(0, 0.04, len(dates)),
        'A': 1.0 + np.random.normal(0, 0.06, len(dates)),
        'BBB': 1.5 + np.random.normal(0, 0.09, len(dates)),
        'BB': 3.0 + np.random.normal(0, 0.15, len(dates)),
        'B': 4.5 + np.random.normal(0, 0.25, len(dates))
    }
    
    # Key economic indicators
    indicators = {
        'CPI': [3.1, 3.0, 3.2, 3.1],
        'Unemployment': [3.9, 3.8, 3.8, 3.7],
        'GDP_Growth': [2.1, 2.0, 2.2, 2.3],
        'Fed_Rate': [5.50, 5.50, 5.50, 5.50]
    }
    
    # News sentiment (mocked)
    news_sentiment = {
        'Treasury_Market': [-0.2, 0.1, 0.3, -0.1, 0.2],
        'Corporate_Bonds': [0.1, -0.3, -0.1, 0.2, 0.1],
        'Fed_Policy': [-0.4, -0.2, 0.1, 0.0, -0.1]
    }
    
    # Create dataframes
    treasury_df = pd.DataFrame(treasuries, index=dates)
    spread_df = pd.DataFrame(corp_spreads, index=dates)
    
    # Recent bond trades
    trades = pd.DataFrame({
        'Bond': ['US Treasury 10Y', 'Apple 3.5% 2028', 'JPM 4.2% 2030', 'T 3.7% 2025', 'GS 4.5% 2027'],
        'Price': [98.25, 97.5, 99.1, 98.7, 97.2],
        'Yield': [4.72, 5.1, 5.3, 4.9, 5.4],
        'Volume': [250000000, 15000000, 12000000, 8000000, 10000000],
        'Trade_Date': [dates[-1], dates[-1], dates[-2], dates[-3], dates[-1]]
    })
    
    return {
        'treasuries': treasury_df,
        'spreads': spread_df,
        'indicators': indicators,
        'news_sentiment': news_sentiment,
        'recent_trades': trades
    }

# Market Analysis LLM Class
class MarketAnalysisLLM:
    def __init__(self, model_name="gpt-4"):
        self.llm = ChatOpenAI(
            model_name=model_name,
            temperature=0.1,
            openai_api_key=os.getenv("OPENAI_API_KEY")
        )
        
        self.system_prompt = """
        You are an expert fixed income market analyst with deep knowledge of bond markets,
        yield curves, credit analysis, and macroeconomic factors. Analyze the provided market data
        and provide insights, risks, and potential trading strategies.
        
        Your analysis should include:
        1. Yield curve analysis - shape, movements, and implications
        2. Credit spread dynamics - widening/tightening and sector analysis
        3. Economic indicators impact on fixed income
        4. Market sentiment interpretation
        5. Trading recommendations with clear rationale
        
        Be specific, data-driven, and provide actionable insights for fixed income traders.
        """
        
    def _format_data(self, market_data):
        """Format market data into a readable string format for the LLM"""
        output = "# FIXED INCOME MARKET DATA ANALYSIS\n\n"
        
        # Treasury yields
        output += "## Treasury Yields (Most Recent)\n"
        recent_date = market_data['treasuries'].index[-1].strftime('%Y-%m-%d')
        treasuries = market_data['treasuries'].iloc[-1].to_dict()
        output += f"Date: {recent_date}\n"
        for tenor, yield_val in treasuries.items():
            output += f"{tenor}: {yield_val:.2f}%\n"
        
        # Treasury yield changes
        output += "\n## Treasury Yield Changes (1 Month)\n"
        yield_changes = market_data['treasuries'].iloc[-1] - market_data['treasuries'].iloc[0]
        for tenor, change in yield_changes.items():
            direction = "up" if change > 0 else "down"
            output += f"{tenor}: {change:.2f}% ({direction})\n"
        
        # Credit spreads
        output += "\n## Corporate Bond Spreads (Most Recent)\n"
        spreads = market_data['spreads'].iloc[-1].to_dict()
        for rating, spread in spreads.items():
            output += f"{rating}: +{spread:.2f}%\n"
        
        # Spread changes
        output += "\n## Corporate Spread Changes (1 Month)\n"
        spread_changes = market_data['spreads'].iloc[-1] - market_data['spreads'].iloc[0]
        for rating, change in spread_changes.items():
            direction = "widened" if change > 0 else "tightened"
            output += f"{rating}: {abs(change):.2f}% ({direction})\n"
        
        # Economic indicators
        output += "\n## Economic Indicators (Last 4 Months)\n"
        for indicator, values in market_data['indicators'].items():
            output += f"{indicator}: {', '.join([str(v) for v in values])}\n"
        
        # Recent trades
        output += "\n## Recent Bond Trades\n"
        for _, row in market_data['recent_trades'].iterrows():
            output += f"Bond: {row['Bond']}, Price: {row['Price']}, Yield: {row['Yield']}%, Volume: ${row['Volume']/1000000}M\n"
        
        # News sentiment
        output += "\n## Market Sentiment (Last 5 Days)\n"
        for segment, scores in market_data['news_sentiment'].items():
            avg_score = sum(scores) / len(scores)
            sentiment = "positive" if avg_score > 0.1 else "negative" if avg_score < -0.1 else "neutral"
            output += f"{segment}: {sentiment} ({avg_score:.2f})\n"
            
        return output
        
    def analyze_market(self, market_data):
        """Analyze market data and generate insights"""
        formatted_data = self._format_data(market_data)
        
        # Create prompt
        prompt = ChatPromptTemplate.from_messages([
            ("system", self.system_prompt),
            ("human", "Please analyze the following fixed income market data:\n\n{market_data}")
        ])
        
        # Create chain
        chain = LLMChain(llm=self.llm, prompt=prompt)
        
        # Run analysis
        response = chain.run(market_data=formatted_data)
        return response
    
    def generate_trade_recommendations(self, market_data, portfolio_context=None):
        """Generate specific trade recommendations"""
        formatted_data = self._format_data(market_data)
        
        portfolio_info = ""
        if portfolio_context:
            portfolio_info = f"\n\nCurrent portfolio information:\n{portfolio_context}"
        
        recommendation_prompt = """
        Based on the market data provided, generate 3-5 specific trade recommendations for a fixed income portfolio.
        For each recommendation:
        1. Specify the exact instrument (including tenor and credit quality)
        2. Indicate buy/sell recommendation
        3. Suggested size/allocation
        4. Risk level (low/medium/high)
        5. Clear rationale tied to market data
        6. Expected return and time horizon
        
        Format as a structured list with clear headers.
        """
        
        prompt = ChatPromptTemplate.from_messages([
            ("system", self.system_prompt),
            ("human", f"Please review the following market data:{portfolio_info}\n\n{formatted_data}\n\n{recommendation_prompt}")
        ])
        
        chain = LLMChain(llm=self.llm, prompt=prompt)
        response = chain.run(market_data=formatted_data)
        return response

# Run the example
def main():
    print("Generating sample fixed income market data...")
    market_data = generate_sample_data()
    
    print("Initializing Market Analysis LLM...")
    analyzer = MarketAnalysisLLM()
    
    print("Running market analysis...")
    analysis = analyzer.analyze_market(market_data)
    print("\n" + "="*80 + "\nMARKET ANALYSIS\n" + "="*80)
    print(analysis)
    
    print("\nGenerating trade recommendations...")
    portfolio_context = """
    Current portfolio composition:
    - US Treasuries: 40% (duration: 5.2 years)
    - Investment Grade Corporate: 35% (avg rating: A-)
    - High Yield Corporate: 15% (avg rating: BB)
    - Agency MBS: 10%
    """
    recommendations = analyzer.generate_trade_recommendations(market_data, portfolio_context)
    print("\n" + "="*80 + "\nTRADE RECOMMENDATIONS\n" + "="*80)
    print(recommendations)
    
    # Bonus: Plot yield curve
    plt.figure(figsize=(10, 6))
    last_day = market_data['treasuries'].iloc[-1]
    first_day = market_data['treasuries'].iloc[0]
    tenors = [2, 5, 10, 30]
    
    plt.plot(tenors, [last_day['2Y'], last_day['5Y'], last_day['10Y'], last_day['30Y']], 'b-o', label='Current')
    plt.plot(tenors, [first_day['2Y'], first_day['5Y'], first_day['10Y'], first_day['30Y']], 'r--o', label='30 Days Ago')
    
    plt.title('Treasury Yield Curve')
    plt.xlabel('Tenor (Years)')
    plt.ylabel('Yield (%)')
    plt.legend()
    plt.grid(True)
    plt.savefig('yield_curve.png')
    print("\nYield curve chart saved as 'yield_curve.png'")

if __name__ == "__main__":
    main()